# Multivariate Normal Dueling Bandit Demo

This notebook demonstrates how to tune the batch size for the multivariate normal dueling bandit using **SciPy's D'Agostino–Pearson K² test** instead of the custom Mardia test used previously.  It also explains each step in the process and shows how to run the bandit algorithm once the batch size is selected.

The general steps are:

1. Load the cost curves, candidate levee heights, posterior parameter draws and mean sea level (MSL) paths.
2. Tune the batch size ``S`` for the central limit theorem by sampling a number of mean cost vectors and applying SciPy's normality test on each coordinate.  We start from a small ``S`` and double it until all p–values exceed our chosen significance threshold.
3. Use the tuned batch size to run the dueling bandit algorithm implemented in ``mvn_dueling_bandit.py``.
4. Inspect the posterior probability that each height is optimal and visualise the convergence over time.

By running this notebook, you can gain confidence that the sample mean cost vectors are approximately multivariate normal and that the bandit algorithm uses a statistically justified stopping rule.

In [2]:
# Imports
import numpy as np
from scipy.stats import normaltest

# Import functions from your project
from optimal_levee_bandit import load_cost_curves, prune_candidates
from mvn_dueling_bandit import _simulate_single_scenario, run_mvn_dueling_bandit

np.set_printoptions(precision=3, suppress=True)

## 1. Load cost curves and parameters

Here we load the damage and protection cost curves for the selected city, define the analysis horizon, and load posterior parameter draws and mean sea level paths.  These objects are required for the Monte Carlo simulation.

In [3]:
from pathlib import Path

# Specify input files
damage_file     = "Damage_cost_curves.tab"
protection_file = "Protection_cost_curves_high_estimate.tab"
city = "Halmstad"

# Load cost curves
heights, damage_costs, protection_costs = load_cost_curves(damage_file, protection_file, city)

# Define analysis horizon (inclusive)
years_range = (2025, 2100)

# Use all heights as candidates (no pruning)
candidate_indices = np.arange(heights.size, dtype=int)

print(f"Loaded {len(heights)} potential levee heights.")

Loaded 25 potential levee heights.


In [4]:
# Load posterior parameter draws and mean sea level paths
# Adjust the filename to match your actual .npz file
pp = np.load("pp_inputs_halmsdad_pp_mixture_2025_2100.npz")
posterior_params = {
    "eta0": pp["eta0"],
    "eta1": pp["eta1"],
    "alpha0": pp["alpha0"],
    "xi": pp["xi"],
    "u": float(pp["u"]),  # threshold in cm
}

# Extract mean sea level paths and corresponding years
years_future   = pp["years_future"]
X_future_paths = pp["X_future_paths"]  # shape (M_pred, T_future) in cm

# Restrict MSL paths to the analysis horizon
start_year, end_year = years_range
mask = (years_future >= start_year) & (years_future <= end_year)
years_slice = years_future[mask]
X_paths_slice_cm = X_future_paths[:, mask]

print(f"MSL paths shape restricted to horizon: {X_paths_slice_cm.shape}")

MSL paths shape restricted to horizon: (20000, 76)


## 2. Tuning the batch size using SciPy's normality test

To apply the multivariate central limit theorem, we want the vector of mean total costs across the candidate heights to be approximately multivariate normal.  One practical way to check this is to apply a **univariate normality test** to each coordinate of the mean vectors.  If each marginal distribution looks normal (as measured by p–values exceeding a significance threshold), the joint distribution is often well approximated by a multivariate normal for large ``S``.

We use the [D'Agostino–Pearson K² test](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.normaltest.html) implemented in `scipy.stats.normaltest`.  For a given batch size ``S``, we sample many cost vectors, average them, and compute the p–value of the omnibus normality test for each coordinate.  If **all** p–values exceed our chosen ``significance`` threshold (e.g., 0.05), we accept this ``S``.  Otherwise we double ``S`` and try again.  This tuning process repeats until acceptance or until a maximum ``S`` is reached.

In [ ]:
def compute_mu_samples(S, rng, test_replicates=200):
    """Compute test_replicates mean cost vectors using batch size S."""
    num_cands = candidate_indices.size
    mu_samples = np.empty((test_replicates, num_cands), dtype=float)
    for r in range(test_replicates):
        # Collect costs for S independent scenarios
        costs_cum = np.empty(num_cands, dtype=float)
        for s in range(S):
            costs_cum= costs_cum+_simulate_single_scenario(
                rng,
                cand_heights=heights[candidate_indices],
                cand_protections=protection_costs[candidate_indices],
                height_grid_m=heights,
                damage_grid=damage_costs,
                X_paths_slice_cm=X_paths_slice_cm,
                posterior_params=posterior_params,
                u_cm=float(posterior_params.get("u", 60.0)),
            )
        mu_samples[r] = costs_cum/S
    return mu_samples

def tune_batch_size_manual(initial_S=100, significance=0.05, test_replicates=200, max_S=1_000_000, rng=None):
    """
    Manually tune S by checking univariate normality with SciPy.

    Returns the tuned batch size and the corresponding mu_samples.
    """
    if rng is None:
        rng = np.random.default_rng()
    S = max(1, int(initial_S))
    while True:
        mu_samples = compute_mu_samples(S, rng, test_replicates)
        # normaltest returns (statistic, p-value) for each column
        try:
            stat, pvals = normaltest(mu_samples, axis=0)
        except Exception:
            # If the test fails, set p-values to zero to force doubling S
            pvals = np.zeros(candidate_indices.size)
            print(f"Tested S={S}: normality test failed, doubling S.")
        min_p = float(np.min(pvals))
        print(f"Tested S={S}: minimum p-value = {min_p:.4f}")
        if np.all(pvals > significance):
            return S, mu_samples
        S *= 1.4
        S = max(1, int(S))
        if S > max_S:
            raise RuntimeError("Failed to achieve approximate normality before reaching max_S.")

# Example usage: tune S with verbose output
rng = np.random.default_rng(12345)
tuned_S, mu_samples = tune_batch_size_manual(initial_S=70, significance=0.05, test_replicates=200, rng=rng)
print(f"Tuned batch size: {tuned_S}")

KeyboardInterrupt: 

In the code cell above, ``tune_batch_size_manual`` repeatedly samples ``test_replicates`` mean cost vectors, applies SciPy's **normaltest** along each dimension, prints the minimum p–value, and doubles ``S`` until all p–values exceed the significance threshold.  Once the condition is met, it returns the tuned batch size and the corresponding sample means.

## 3. Run the multivariate normal dueling bandit

Now that we have a tuned batch size, we can run the dueling bandit.  The function `run_mvn_dueling_bandit` encapsulates the entire adaptive procedure: it reuses the normality tuning procedure internally (so we set ``initial_S=tuned_S`` to avoid starting from scratch), computes a multivariate normal prior, and updates the posterior as new floods are simulated.  The algorithm stops when the posterior probability that some height is optimal exceeds ``1 − delta``.

In [ ]:
delta = 0.05  # target misselection probability
rng = np.random.default_rng(98765)

# Run the bandit algorithm with the tuned initial_S
best_height, history = run_mvn_dueling_bandit(
    heights=heights,
    damage_costs=damage_costs,
    protection_costs=protection_costs,
    candidate_indices=candidate_indices,
    years_all=years_future,
    X_pred_paths_cm=X_future_paths,
    posterior_params=posterior_params,
    years_range=years_range,
    delta=delta,
    initial_S=tuned_S,    # use the manually tuned S as a starting point
    significance=0.05,
    test_replicates=200,
    post_samples=2000,
    max_rounds=50_000,
    check_every=1_000,
    rng=rng,
    verbose=True,
)

print(f"Selected best design height: {best_height:.2f} m")

## 4. Inspect the convergence history

The `history` dictionary returned by `run_mvn_dueling_bandit` contains the rounds at which the stopping rule was evaluated, the index and value of the current best height, and the posterior probability `p_best` that this candidate is optimal.  We can convert this to a Pandas DataFrame and plot the probability of the best height and the evolution of the selected height over time.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Build DataFrame
hist_df = pd.DataFrame(history)
display(hist_df.tail())

# Plot posterior probability of the best height
fig, ax1 = plt.subplots()
ax1.plot(hist_df['rounds'], hist_df['p_best'], marker='o')
ax1.axhline(1 - delta, color='red', linestyle='--', label=f'1 - delta = {1 - delta}')
ax1.set_xlabel('Rounds')
ax1.set_ylabel('Posterior prob. best')
ax1.set_title('Convergence of the posterior probability for the best height')
ax1.legend()
plt.show()

# Plot evolution of the empirical best height
fig, ax2 = plt.subplots()
ax2.step(hist_df['rounds'], hist_df['best_height'], where='post', marker='o')
ax2.set_xlabel('Rounds')
ax2.set_ylabel('Height (m)')
ax2.set_title('Evolution of the empirically best height')
plt.show()